In [2]:
import numpy as np
inf=float("inf")

import pandas as pd

import matplotlib.pyplot as plt
plt.style.use('ggplot')

import scipy.stats as st

import stablerank.srank as sr
import stablerank.geometry_objects_2020 as ge

from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
import _pickle as pickle
import csv

from ripser import ripser
from scipy.cluster.hierarchy import dendrogram, linkage, fcluster

from PIL import Image
from ast import literal_eval

### Import Training Data - Option 1

In [19]:
#df=pd.read_csv('challenges-in-representation-learning-facial-expression-recognition-challenge/icml_face_data.csv', sep=',')
df=pd.read_csv('challenges-in-representation-learning-facial-expression-recognition-challenge/train.csv', sep=',')

In [21]:
# Removing duplicate values and reindexing
df.drop_duplicates(subset =["pixels"], inplace = True)
df=df.reindex(range(0,len(df)-1))

### Import Test Data - Option 2

In [4]:
df=pd.read_csv('challenges-in-representation-learning-facial-expression-recognition-challenge/test.csv', sep=',')

In [5]:
# Removing duplicate values and reindexing
df.drop_duplicates(subset =["pixels"], inplace = True)
df=df.reindex(range(0,len(df)-1))

## Try for one image

In [24]:
# Take the first picture as an example
pixel_values = [int(numeric_string) for numeric_string in df["pixels"][3].split(' ')]

image_dummy = np.empty((48*48,3), int)
for i in range(0,len(image_dummy)):
    image_dummy[i] = [(100/47)*(i-i%48)/48, (100/47)*(i%48), (100/47)*(47*pixel_values[i]/255)]

image_stacked = np.vstack(image_dummy)
image = sr.EucObject(image_stacked)

In [26]:
index = 300
degree="H1"
md=1

# Compute the stable rank as above
pixel_values_ti = [int(numeric_string) for numeric_string in df["pixels"][index].split(' ')]
image_dummy_ti = np.empty((48*48,3), int)
for i in range(0,len(image_dummy_ti)):
    image_dummy_ti[i] = [(100/47)*(i-i%48)/48, (100/47)*(i%48), (100/47)*(47*pixel_values_ti[i]/255)]

image_stacked_ti = np.vstack(image_dummy_ti)
image_ti = sr.EucObject(image_stacked_ti)

b_ti = image.get_bc(maxdim=md)
S_ti = sr.bc_to_sr(b_ti, degree)

content = S_ti.content

contents_1 = []
contents_1.append(content[0])

contents_2 = []
contents_2.append(content[1])

indeces = []
indeces.append(index)

data_test = {'old index':indeces, 'content 1':contents_1, 'content 2':contents_2}
df_out_test = pd.DataFrame(data_test)

In [27]:
df_out_test.to_csv("Results/test.csv")

In [28]:
df_reload=pd.read_csv("Results/test.csv")

### Reconstructing the PCNIF from the content

In [34]:
c1_temp = df_reload["content 1"][0]
c1 = [float(x) for x in c1_temp[1:-1].split()]
c2_temp = df_reload["content 2"][0]
c2 = [float(x) for x in c2_temp[1:-1].split()]
ce = [c1, c2]
#### Result
res = sr.Pcnif(ce)

## Exporting for one emotion

In [39]:
# Selecting one emotion
e = 3
#0=Angry, 1=Disgust, 2=Fear, 3=Happy, 4=Sad, 5=Surprise, 6=Neutral

# specifying the number of pictures for subsampling
number_of_pictures = 100

# select homology and maxdim
degree="H0"
md=0

# Set up empty arrays for the indices and contents
indices = []
contents_1 = []
contents_2 = []

# Select random images for subsampling for emotion e
pos = np.array(np.where(df["emotion"]==e))[0]
rand_pos = pos[list(np.random.permutation(np.arange(0,len(pos)-1))[:number_of_pictures])]
    
# Calculate stable ranks
for l in range(0,number_of_pictures):
    pixel_values_it = [int(numeric_string) for numeric_string in df["pixels"][rand_pos[l]].split(' ')]
    image_dummy_it = np.empty((48*48,3), int)
    for i in range(0,len(image_dummy_it)):
        image_dummy_it[i] = [(100/47)*(i-i%48)/48, (100/47)*(i%48), (100/47)*(47*pixel_values_it[i]/255)]

    image_stacked_it = np.vstack(image_dummy_it)
    image_it = sr.EucObject(image_stacked_it)

    b_it = image_it.get_bc(maxdim=md)
    S_it = sr.bc_to_sr(b_it, degree)
    
    content = S_it.content
    
    indices.append(rand_pos[l])
    contents_1.append(content[0])
    contents_2.append(content[1])
    
data = {'old index':indices, 'content 1':contents_1, 'content 2':contents_2}
df_out = pd.DataFrame(data)
df_out.to_csv("Results/")